In [ ]:
from bs4 import BeautifulSoup as BSHTML
import pandas as pd
from urllib.request import urlopen

In [ ]:
df = pd.DataFrame()
data_dict = {}

author_urls = {
    'Chimamanda-Ngozi-Adichie': "Chimamanda-Ngozi-Adichie/e/B00PODW5UG",
    # 'Elise Bohan': "https://www.amazon.com/Elise-Bohan/e/B09X6CPCYR"
}

# Get the HTML source

author_url_prefix = 'https://www.amazon.com/'
author = 'Chimamanda-Ngozi-Adichie'
page = urlopen(author_url_prefix + author_urls[author])
soup = BSHTML(page)

In [ ]:

data_dict[author] = {}

# This dictionary structure - 
# key - the name of the author
# value is a dictionary again 
# dictionary key - name of the recommended author, img_url, amazon page url

In [ ]:
# Get the list of authors, and their image URLs
# from the soup. 

rec_authors = []
auth_img = []
for item in soup.find_all(class_="authorListImage"):
    rec_auth = item["alt"]
    rec_authors.append(rec_auth)
    
    data_dict[author][rec_auth] = []
    data_dict[author][rec_auth].append(item["src"])


# data_dict

In [ ]:
# The soups gives a lot of links; need to collect only the ones with author names
# in certain format. Get that format first.
auth_links = []
auth_link_names = {}
for auth in rec_authors:
    auth_link_names[auth] = ("-".join(n.strip().replace('.', '') for n in auth.split()))

for item in soup.findAll(class_="a-link-normal"):
    if '/e/' in item["href"]:
        link = item["href"]
        idx = link.find('?ref=')
        if idx > 0:
            link = link[:idx]
        auth_links.append(link)

auth_links = list(set(auth_links))

for rec_auth, link in auth_link_names.items():
    for auth_link in auth_links:
        if link in auth_link:
            data_dict[author][rec_auth].append(auth_link)
            author_urls[rec_auth] = auth_link

# This will ensure that for all the recommended authors, their links are found
# assert(len(auth_link_names) == 0)



In [ ]:
for item in soup.find('span', {'id': 'author_biography'}):
    print (item.text.strip())

In [ ]:
df = pd.DataFrame.from_dict(data_dict[author], orient='index').reset_index()

In [ ]:
df.columns = ['recommended_author', 'img_url', 'author_url']
df.loc[:, 'author'] = author
columns = ['author', 'recommended_author', 'img_url', 'author_url'] 
df[columns]